### Step 1: inculde the setup of source dataset, metadata and macros

In [ ]:
*---------------------------------------------------------------*;
* ADAE.sas creates the ADaM ADAE-structured data set
* for AE data (ADAE), saved to the ADaM libref.
*---------------------------------------------------------------*;

%include "/home/junmeng0/02-Implementing CDISC Using SAS/chapter07/setup.sas";

#### 1.1 Macro: setup.sas

In [ ]:
**** defines common librefs and SAS options.;

%let sasroot=%sysfunc(sysget(SASROOT));
%let sysin = %sysfunc(getoption(sysin));

options ls=256 nocenter extendobscounter=NO
        mautosource 
        sasautos=("/home/junmeng0/02-Implementing CDISC Using SAS/macro") ;
        
                    
%include "/home/junmeng0/02-Implementing CDISC Using SAS/macro/xpt_macros.sas";
libname sdtm    "/home/junmeng0/02-Implementing CDISC Using SAS/chapter03";
libname adam    "/home/junmeng0/02-Implementing CDISC Using SAS/chapter07";


proc format;
        value _0n1y 0 = 'N'
                    1 = 'Y'
        ;                    
        value avisitn 1 = '3'
                      2 = '6'
        ;                      
        value popfl 0 - high = 'Y'
                    other = 'N'
        ;                    
        value $trt01pn  'Analgezia HCL 30 mg' = '1'
                        'Placebo'             = '0'
        ;
        value agegr1n 0 - 54 = "1"
                      55-high= "2"
        ;                      
        value agegr1_ 1 = "<55 YEARS"
                      2 = ">=55 YEARS"
        ;                      
        value $aereln  'NOT RELATED'        = '0'
                       'POSSIBLY RELATED'   = '1'
                       'PROBABLY RELATED'   = '2'
        ;
        value $aesevn  'MILD'               = '1'
                       'MODERATE'           = '2'
                       'SEVERE'             = '3'
        ;                                              
        value relgr1n 0 = 'Not related'
                      1 = 'Related'
        ;                       
        value evntdesc 0 = 'PAIN RELIEF'
                       1 = 'PAIN WORSENING PRIOR TO RELIEF'
                       2 = 'PAIN ADVERSE EVENT PRIOR TO RELIEF'
                       3 = 'COMPLETED STUDY PRIOR TO RELIEF'
        ;                    
run;
 

### Step2. Create empty ADAE dataset

In [ ]:
**** CREATE EMPTY ADAE DATASET CALLED EMPTY_ADAE;
options mprint ;*symbolgen;
%let metadatafile=/home/junmeng0/02-Implementing CDISC Using SAS/chapter07/ADAM_METADATA.xls;

%make_empty_dataset(metadatafile=&metadatafile,dataset=ADAE);

#### 2.1 macro make_empty_dataset

In [ ]:
/*---------------------------------------------------------------*;
 make_empty_dataset.sas creates a zero record dataset based on a  dataset metadata spreadsheet.  The dataset created is called * EMPTY_** 
 where "**" is the name of the dataset.  
This macro also creates a global macro variable called **KEEPSTRING that holds the dataset variables desired and listed in the order they should appear.  [The variable order is dictated by VARNUM in the metadata spreadsheet

MACRO PARAMETERS:
- metadatafile = the MS Excel file containing the dataset metadata
- dataset = the dataset or domain name you want to extract
*---------------------------------------------------------------*/;
%macro make_empty_dataset(metadatafile=,dataset=);

    proc import 
        datafile="&metadatafile"
        out=_temp 
        dbms=xls
        replace;
        sheet="VARIABLE_METADATA";
    run;

    ** sort the dataset by expected specified variable order;
    proc sort
      data=_temp;
	  where domain = "&dataset";
        by varnum;	  
    run;

    ** create keepstring macro variable and load metadata 
    ** information into macro variables;
    %global &dataset.KEEPSTRING;
    data _null_;
      set _temp nobs=nobs end=eof;

        if _n_=1 then
          call symput("vars", compress(put(nobs,3.)));
    
        call symputx('var'    || compress(put(_n_, 3.)), variable);
        call symputx('label'  || compress(put(_n_, 3.)), label);
        call symputx('length' || compress(put(_n_, 3.)), put(length, 3.));


        ** valid ODM types include TEXT, INTEGER, FLOAT, DATETIME, 
        ** DATE, TIME and map to SAS numeric or character;
        if upcase(type) in ("INTEGER", "FLOAT") then
          call symputx('type' || compress(put(_n_, 3.)), "");
        else if upcase(type) in ("TEXT", "DATE", "DATETIME", "TIME") then
          call symputx('type' || compress(put(_n_, 3.)), "$");
        else
          put "ERR" "OR: not using a valid ODM type.  " type=;


        ** create **KEEPSTRING macro variable;
        length keepstring $ 32767;	 
        retain keepstring;		
        keepstring = compress(keepstring) || "|" || left(variable); 
        if eof then
          call symputx(upcase(compress("&dataset" || 'KEEPSTRING')), 
                       left(trim(translate(keepstring," ","|"))));
    run;
     

    ** create a 0-observation template data set used for assigning 
    ** variable attributes to the actual data sets;
    data EMPTY_&dataset;
        %do i=1 %to &vars;           
           attrib &&var&i label="&&label&i" 
             %if "&&length&i" ne "" %then
               length=&&type&i.&&length&i... ;
           ;
           %if &&type&i=$ %then
             retain &&var&i '';
           %else
             retain &&var&i .;
           ;
        %end;
        if 0;
    run;

%mend make_empty_dataset;


In [ ]:
proc sort
  data = adam.adsl
  (keep = usubjid siteid country age agegr1 agegr1n sex race trtsdt trt01a trt01an saffl)
  out = adsl;
    by usubjid;

In [ ]:
data adae;
  length relgr1 $15.;
  merge sdtm.ae (in = inae) adsl (in = inadsl);
    by usubjid ;
    
        if inae and not inadsl then
          put 'PROB' 'LEM: Subject missing from ADSL?-- ' usubjid= inae= inadsl= ;
        
        rename trt01a    = trta
               trt01an   = trtan
        ;               
        if inadsl and inae;
        
        %dtc2dt(aestdtc, prefix=ast, refdt=trtsdt);
        %dtc2dt(aeendtc, prefix=aen, refdt=trtsdt);

        if index(AEDECOD, 'PAIN')>0 or AEDECOD='HEADACHE' then
          CQ01NAM = 'PAIN EVENT';
        else
          CQ01NAM = '          ';
          
        aereln = input(put(aerel, $aereln.), best.);
        aesevn = input(put(aesev, $aesevn.), best.);
        relgr1n = (aereln>0); ** group related events (AERELN>0);
        relgr1  = put(relgr1n, relgr1n.);
        
        * Event is considered treatment emergent if it started on or after ;
        * the treatment start date.  Assume treatment emergent if the start;
        * date is missing (and the end date is either also missing or on or;
        *  after the treatment start date)                                 ;
        trtemfl = put((astdt>=trtsdt or (astdt<=.z  and not(.z<aendt<trtsdt))), _0n1y.);
        if astdt>=trtsdt then
          trtemfl = 'Y';
        format astdt aendt yymmdd10.;
run;






In [ ]:
** assign variable order and labels;
data adae;
  retain &adaeKEEPSTRING;
  set EMPTY_adae adae;
run;

In [ ]:
**** SORT adae ACCORDING TO METADATA AND SAVE PERMANENT DATASET;
%make_sort_order(metadatafile=&metadatafile, dataset=ADAE);

In [ ]:
proc sort
  data=adae(keep = &adaeKEEPSTRING)
  out=adam.adae;
    by &adaeSORTSTRING;
run;        

